## Machine Learning Project: Playlist Generator using Machine Learniing

Process
1. Understand problem and method of attack


In [54]:
import numpy as np
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint


In [3]:
client_ID = 'fe5b05a7f1544404a53c4ba35ebcbb1a'
client_SECRET = 'cc7283ec42534341b242278e56377254'

In [9]:

scope = 'user-library-read'

client_credentials_manager = SpotifyClientCredentials(client_id=client_ID, client_secret=client_SECRET)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)


spotify_ID ='4pqM1yOZK6JkjUHWIUMmab' #playlist_link = https://open.spotify.com/playlist/4pqM1yOZK6JkjUHWIUMmab?si=1cd401ecf53a429c
#spotify_ID ='31maatcac72uzwwtlzaz5jnszmxe' #playlist_link = https://open.spotify.com/user/31maatcac72uzwwtlzaz5jnszmxe?si=cc53545e2dd34226

track_uris = []

for i in [0,100,200,300,400,500,600]:
    for x in sp.playlist_items(spotify_ID,limit=100, offset=i)["items"]:
        
        y = x["track"]["uri"]
        track_uris.append(sp.audio_features(y)[0])


df = pd.DataFrame(track_uris)
df.drop(['type', 'id', 'uri', 'track_href', 'analysis_url'], axis='columns', inplace=True)
df.head(10)

df.to_csv("Dataset/user_playlist.csv")



In [39]:
spotify_df = pd.read_csv("Dataset/dataset.csv")
spotify_df.drop(["Unnamed: 0", "track_id", "artists", "album_name", "track_name", "popularity", "explicit", "track_genre"], axis="columns", inplace=True)

cols = spotify_df.columns.tolist()
order = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 0, 12]
new_cols = [cols[i] for i in order]

spotify_df = spotify_df[new_cols]
spotify_df['label'] = 0
spotify_df = spotify_df.sample(400)



In [35]:
library_df = pd.read_csv("Dataset/user_playlist.csv")
library_df.drop("Unnamed: 0", axis="columns", inplace=True)
library_df['label'] = 1
library_df.head(10)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,label
0,0.807,0.893,11,-3.745,0,0.0347,0.0451,0.000028,0.3660,0.537,126.011,230747,4,1
1,0.798,0.676,6,-5.041,1,0.0442,0.0902,0.000000,0.0894,0.843,101.956,219947,4,1
2,0.788,0.859,2,-2.724,1,0.0856,0.2810,0.000000,0.0424,0.822,141.020,207853,4,1
3,0.763,0.661,4,-5.592,0,0.0560,0.1690,0.000000,0.1170,0.632,95.002,153627,4,1
4,0.599,0.448,8,-6.312,1,0.0232,0.1630,0.000000,0.1060,0.168,95.050,263400,3,1
5,0.715,0.620,1,-6.005,0,0.0484,0.4170,0.000000,0.0822,0.172,97.950,221520,4,1
6,0.607,0.649,11,-6.695,0,0.0362,0.0306,0.000025,0.1740,0.505,102.996,247627,4,1
7,0.640,0.647,5,-8.114,0,0.1860,0.1240,0.000000,0.0801,0.546,92.979,197867,4,1
8,0.591,0.764,1,-5.484,1,0.0483,0.0383,0.000000,0.1030,0.478,169.928,141806,4,1
9,0.713,0.676,10,-5.483,1,0.0270,0.0377,0.000000,0.1540,0.721,99.048,227395,4,1


In [42]:
data = [library_df, spotify_df]
combine_df = pd.concat(data)
combine_df.to_csv("Dataset/final.csv")

In [55]:
X = combine_df.drop('label', axis="columns")
y = combine_df['label']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)




Accuracy: 0.6643356643356644


In [76]:
top_100 ='6UeSakyzhiEt4NB3UAd6NQ' #playlist_link = https://open.spotify.com/playlist/6UeSakyzhiEt4NB3UAd6NQ?si=745d5d06e51c444b

top100_uris = []
for x in sp.playlist_items(top_100)["items"]:
    y = x["track"]["uri"]
    top100_uris.append(sp.audio_features(y)[0])


final_df = pd.DataFrame(top100_uris)
result_df = final_df.copy()

final_df = final_df.drop(['type', 'id', 'uri', 'track_href', 'analysis_url'], axis='columns')

y_pred = rf.predict(final_df)

result_df['results'] = y_pred
result_df = result_df[['results','uri']]
result_df = result_df[result_df.results == 1]
result_df.shape



,results,uri
1,1,spotify:track:7K3BhSpAxZBznislvUMVtn
3,1,spotify:track:2dHHgzDwk4BJdRwy9uXhTO
4,1,spotify:track:7oDd86yk8itslrA9HRP2ki
6,1,spotify:track:02Zkkf2zMkwRGQjZ7T4p8f
9,1,spotify:track:4686eQ81DEswHa90bcdlC9
10,1,spotify:track:6NHpyYvJyQsg2nXXzGYc2R
14,1,spotify:track:4LRPiXqCikLlN15c3yImP7
15,1,spotify:track:4uUG5RXrOk84mYEfFvj3cK
19,1,spotify:track:0PAcdVzhPO4gq1Iym9ESnK
22,1,spotify:track:0JXXNGljqupsJaZsgSbMZV
